### Conversation Q/A RAG 
1. simple RAG app doestn't focus on previous questions(no chat memory )
2. convo RAG has some sort of "memory" of past questions and answers, and some logic for incorporating those into its current thinking.

- focusing on adding logic for incorporating historical messages using 2 approaches-->1. chains, 2. Agents

In [51]:
# to load envi variables
import os
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
# using groq for llm 

groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192",groq_api_key=groq_api_key,max_tokens=40000)

In [53]:
# importing modules
import bs4 # web scraping
from langchain_community.document_loaders import WebBaseLoader # to load webpage content
from langchain.text_splitter import RecursiveCharacterTextSplitter # to split text 
from langchain_chroma import Chroma # to store vectors in db
from langchain_core.prompts import ChatPromptTemplate # to define prompt for llm
from langchain.chains import create_retrieval_chain #to create retrieval chain 
from langchain.chains.combine_documents import create_stuff_documents_chain # Create a chain for passing a list of Documents to a model.

# for free embeddings using hugging face models
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings # to embed text to vectors
HF_embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


c:\Users\Admin\Desktop\Langchain\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [54]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [55]:
# split embed and vectorspace
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=300)

splitted_docs = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=docs,embedding=HF_embedding)

retriever = vectorstore.as_retriever()


In [56]:
# prompts
# system prompt is telling llm how should it act/work
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ])


In [57]:
# chains
question_answer_chain = create_stuff_documents_chain(llm, prompt)
# here prompt requires context in system-prompt create_stuff_doc_chain combines all the docs and sends it to llm

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# this retrieval chain retrieves docs and then passes question_ans_chain-- here docs are context and prompt sent to llm

In [58]:
# output from llm 
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01j96bvyvve20t6wjmry5xffgc` on tokens per minute (TPM): Limit 30000, Used 0, Requested 43311. Please try again in 26.622s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}